<div align="center">
    <H1>Aprendizado de Máquina - Trabalho Final</H1>
    <H3>Prof.º Daniel Roberto Cassar</H3> 
</div>

<br>

<div align="right">
    <H3>Guilda: Carcajus</H3>
    <H4>Aniel Souza Ribeiro Neto</H4>
    <H4>Caio Cogo Beriam</H4>
    <H4>Joaquim Junior Ferola Fonseca</H4>
</div>

<br> 

<div align="center">
    <img src="Imagens/carcajus_bg.png" width="500"/>
</div>

# Lore

# Introdução

A área de nanotoxicologia, que estuda como nanomateriais interagem com sistemas biológicos, tem se tornado cada vez mais importante com os avanços da nanotecnologia, permitindo que esses materiais sejam implementados de maneira segura ao meio ambiente e aos possíveis seres vivos que venham a entrar em contato com esses nanomateriais. Entretanto, como outras características dos nanomateriais, sua toxicidade pode variar muito conforme a morfologia do composto, assim, para ser analisada corretamente uma série de fatores devem ser levados em consideração, o que torna difícil a previsão da toxicicidade dos materiais. 

Sob essa perspectiva, nosso trabalho tem o objetivo de facilitar esse trabalho através do treinamento de um modelo de Aprendizado de Máquina Supervisionado capaz de prever se uma nanopartícula é tóxica com base em características da sua morfologia. Para isso, utilizaremos como fundamento um conjunto de dados contendo as características de diversas nanoparticulas e se elas são ou não tóxicas. O dataset foi elaborado pelo trabalho Subramanian NA, Palaniappan A. NanoTox: Development of a Parsimonious In Silico Model for Toxicity Assessment of Metal-Oxide Nanoparticles Using Physicochemical Features. ACS Omega 2021, 6, 17, 11729–11739 doi:10.1021/acsomega.1c01076, e o obtivemos pela plataforma Kaggle através do endereço https://www.kaggle.com/datasets/apalania/toxicityassessment-meoxnp.

Dessa forma, este trabalho será dividido em etapas para melhor organização e didática, assim, ao longo deste notebook passaremos pelas seguintes etapas: 
1. Preparo dos dados: Nessa seção será feita a obtenção e tratamento dos dados, permitindo que eles sejam posteriormente utilizados sem problemas, além disso, será feita também a divisão dos dados que serão utilizados para treino e teste dos modelos;
2. Estudo dos modelos: Nessa etapa faremos o estudo de 5 modelos através do módulo optuna para optimzação de hiperparâmetros e estratégias de normalização de dados e tratamento de atributos, buscando o melhor pipeline encontrado pelo optuna para cada modelo;
3. Teste dos modelos: Após definirmos os melhores pipelines encontrados pelo optuna, iremos instânciar, treinar e comparar os 5 pipelines em busca do de melhor desempenho, que, por fim, utilizaremos para prever os dados de teste;
4. Resultados e conclusão: Enfim, discutiremos os resultados obtidos pelo trabalho e apresentaremos sua conclusão.

# Módulos utilizados

In [1]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.feature_selection import RFE
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from optuna import create_study
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

# Preparo dos dados

## Importando o dataset

In [2]:
df = pd.read_csv("nanotox_dataset.tsv", sep="\t")
df

,NPs,coresize,hydrosize,surfcharge,surfarea,Hsf,Ec,Ev,MeO,Cellline,...,ratio,e,esum,esumbyo,MW,NMetal,NOxygen,ox,viability,class
0,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,HCMEC,...,5.556,1.61,3.22,1.073,101.96,2,3,3,92.5258,nonToxic
1,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,HCMEC,...,5.556,1.61,3.22,1.073,101.96,2,3,3,96.1340,nonToxic
2,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,HCMEC,...,5.556,1.61,3.22,1.073,101.96,2,3,3,93.5567,nonToxic
3,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,HCMEC,...,5.556,1.61,3.22,1.073,101.96,2,3,3,97.6804,nonToxic
4,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,HCMEC,...,5.556,1.61,3.22,1.073,101.96,2,3,3,94.8454,nonToxic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,ZnO,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,Caco2,...,2.703,1.65,1.65,1.650,81.38,1,1,2,127.4363,nonToxic
479,ZnO,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,Caco2,...,2.703,1.65,1.65,1.650,81.38,1,1,2,116.3751,nonToxic
480,ZnO,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,Caco2,...,2.703,1.65,1.65,1.650,81.38,1,1,2,40.8796,Toxic
481,ZnO,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,Caco2,...,2.703,1.65,1.65,1.650,81.38,1,1,2,86.8566,nonToxic


## Tratamento dos dados

### $\bullet$ Removendo linhas com dados faltantes ou duplicados

In [3]:
# Removendo linhas com valores NaN
df = df.dropna()
# Removendo linhas duplicadas
df = df.drop_duplicates()

### $\bullet$ Tranformando atributos categóricos em numéricos através da codificação one-hot

In [4]:
# Encontrando as colunas categóricas do dataset
colunas_categoricas = list(df.columns[df.dtypes == 'object'])
print(colunas_categoricas)

['NPs', 'Cellline', 'Celltype', 'class']


In [5]:
# Aplicando o one-hot encoding nas colunas categóricas
df = pd.get_dummies(df,
                    columns=colunas_categoricas[:-1],
                    dtype="int")
df

,coresize,hydrosize,surfcharge,surfarea,Hsf,Ec,Ev,MeO,Expotime,dosage,...,Cellline_ChangLiv,Cellline_HCMEC,Cellline_HONDC,Cellline_L02,Cellline_MCF10A,Cellline_SHSY5Y,Cellline_SW480,Cellline_WI38,Celltype_Cancer,Celltype_Normal
0,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,0.001,...,0,1,0,0,0,0,0,0,0,1
1,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,0.010,...,0,1,0,0,0,0,0,0,0,1
2,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,0.100,...,0,1,0,0,0,0,0,0,0,1
3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,1.000,...,0,1,0,0,0,0,0,0,0,1
4,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,5.000,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,48,1.000,...,0,0,0,0,0,0,0,0,1,0
479,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,48,10.000,...,0,0,0,0,0,0,0,0,1,0
480,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,48,100.000,...,0,0,0,0,0,0,0,0,1,0
481,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,24,0.100,...,0,0,0,0,0,0,0,0,1,0


### $\bullet$ Removendo colunas com variância 0

In [6]:
# Instânciando o VarianceThreshold 
threshold = VarianceThreshold(threshold=0.0)
# Ajustando o modelo e transformando os dados
threshold.fit_transform(df.loc[:, df.dtypes != 'object'])
df

,coresize,hydrosize,surfcharge,surfarea,Hsf,Ec,Ev,MeO,Expotime,dosage,...,Cellline_ChangLiv,Cellline_HCMEC,Cellline_HONDC,Cellline_L02,Cellline_MCF10A,Cellline_SHSY5Y,Cellline_SW480,Cellline_WI38,Celltype_Cancer,Celltype_Normal
0,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,0.001,...,0,1,0,0,0,0,0,0,0,1
1,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,0.010,...,0,1,0,0,0,0,0,0,0,1
2,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,0.100,...,0,1,0,0,0,0,0,0,0,1
3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,1.000,...,0,1,0,0,0,0,0,0,0,1
4,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,5.000,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,48,1.000,...,0,0,0,0,0,0,0,0,1,0
479,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,48,10.000,...,0,0,0,0,0,0,0,0,1,0
480,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,48,100.000,...,0,0,0,0,0,0,0,0,1,0
481,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,24,0.100,...,0,0,0,0,0,0,0,0,1,0


### $\bullet$ Tranformando o target para a forma numérica

In [7]:
df["class"] = df["class"].map({'nonToxic': 0, 'Toxic': 1})

df["class"]

0      0
1      0
2      0
3      0
4      0
      ..
478    0
479    0
480    1
481    0
482    0
Name: class, Length: 477, dtype: int64

## Separando os dados para treino e teste

In [8]:
# Definindo a semente aleatória
semente = 3931421

# Definindo a lista de atributos
atributos = df.drop(columns=["class"])
# Definindo o target
target = df["class"]

# Realizando a separação dos dados 
x_treino, x_teste, y_treino, y_teste = train_test_split(
    atributos,
    target,
    test_size=0.1,
    random_state=semente,
    stratify=df["class"]
)

# Estudos dos modelos 

Nessa etapa será utilizada o módulo optuna para optimização de hiperparâmetros e escolha de estratégias de normalização dos dados para 5 modelos: Um KNN, uma SVM, uma Regressão Logística, uma DecisionTree e uma RandomFlorest, todos de classificação e da biblioteca scikit-learn. Para cada modelo mencionado, o processo realizado seguirá o seguinte fluxo: 
1. Criar a função para instânciar os modelos: A função que possui os critérios que o optuna testará, retornando o modelo com os parâmetros definidos pelo optuna.
2. Criar a função que será otimizada pelo optuna: A função que computará a métrica a ser minimizada ou maximizada pelo optuna.
3. Criar um estudo do optuna: Arquivo onde serão salvos os testes e resultados do optuna.
4. Indicar testes desejados: Indicar testes com valores específicos que desejamos que o optuna teste.
5. Criar a função objetivo parcial: Função que retorna a função a ser otimizada, necessária para o optuna.
6. Rodar o optuna e otimizar os parâmetros.

Vamos entender um pouco mais afundo cada uma das etapas, lembrando que serão realizadas para cada um dos modelos: 
1. A primeira etapa é a criação de uma função que possui em seu corpo opções de escolha que serão testadas pelo optuna. Neste trabalho, cada um dos modelos terá uma função que testa diferentes valores para seus hiperparâmetros, que sejam relevantes, que serão salvos em um dicionário para serem utilizados para instanciar os modelos posteriormente. Depois, o optuna poderá escolher entre normalizar ou não dados, seguindo as normalizações padrão, por mínimos e máximos ou máximo absoluto, e também poderá escolher entre aplicar ou não estratégias de redução de dimensionalidade ou seleção de atributos, adicionando as estratégias escolhidas a uma lista de passos para a criação de um pipeline. Por fim, será instânciado um modelo com os parâmetros definidos e criaremos um pipeline com os passos de tratamento de dados, caso escolhidos pelo optuna, e o modelo instanciado, que será retornado pela função.

2. Aqui será criada uma função que chama a função para intanciar o modelo e testa o modelo criado através de uma validação cruzada seguindo uma estratégia de kfold estratificado com 3 folds e métrica f1-score. A função retorna o desempenho médio do modelo.

3. Criaremos nessa etapa o estudo do optuna, que gera um arquivo contendo os dados obtidos pela optimização que será feita posterioremente, caso já haja um arquivo do estudo ele será lido ao invés de outro ser criado. Nesse ponto, caso deseje utilizar os resultados já obtidos basta baixar os estudos apresentados no repositório deste trabalho no github. O estudo irá buscar maximizar o valor recebido pela função objetivo ao longo das iterações.

4. Agora indicaremos alguns casos particulares que desejamos analisar. Vamos indicar que o optuna teste ao menos uma vez cada caso de normalização e tratamento de dados com os parâmetros base do sklearn.

5. O optuna requer que a função objetivo seja chamada por uma outra função, portanto, criamos uma função que retorna a função objetivo com o trial do optuna.

6. Por fim, faremos optimização de parâmetros pelo optuna. Note que a linha que faz essa etapa está na forma de comentário por padrão pois considera que os estudos disponibilizados no repositório tenham sido baixados, caso tenha optado por não baixar os estudos ou tenha interesse em fazer mais estudos, basta remover a hashtag `#` no início da linha para realizar a optimização.

## Otimizando um modelo K-NearestNeighbors Classifier 

### $\bullet$ Criando a Função para Instanciar o Modelo



In [9]:
def instanciador_knn(trial): 
    """Recebe um trial do optuna e retorna uma instância de um modelo com classificador KNN"""

    # Definindo os parâmetros do modelo 
    params = {
        "n_neighbors": trial.suggest_int("n_neighbors", 1, 200, log=True),
        "p": trial.suggest_int("p", 1, 2),
        "weights": trial.suggest_categorical("weights", ["uniform", "distance"])
    }

    # Lista de etapas para o pipeline 
    steps = []

    # Definindo a estratégia de normalização 
    normalization = trial.suggest_categorical("normalization", [None, "standard", "minmax", "maxabs"])

    # Adiciona normalização padrão
    if normalization == "standard": 
        steps.append(StandardScaler())
    # Adiciona normalização por máximos e mínimos
    elif normalization == "minmax": 
        steps.append(MinMaxScaler())
    # Adiciona normalização por máximo absoluto
    elif normalization == "maxabs": 
        steps.append(MaxAbsScaler())

    # Definindo estratégia de redução de dimensionalidade ou seleção de atributos 
    treatment = trial.suggest_categorical("treatment", [None, "pca"])

    # Adiciona tratamento PCA 
    if treatment == "pca": 
        # Definindo o número de componentes a serem mantidas pelo pca
        components = trial.suggest_int("pca_components", 2, 38)
        steps.append(PCA(n_components=components))

    # Instânciando o modelo
    modelo = KNeighborsClassifier(**params)
    steps.append(modelo)

    # Criando o pipeline
    pipeline = make_pipeline(*steps)

    return pipeline

### $\bullet$ Criando a função objetivo do optuna

In [10]:
def funcao_objetivo_knn(trial, X_treino, Y_treino): 
    """Função a ser otimizada pelo Optuna"""
    # Instânciando o modelo 
    modelo = instanciador_knn(trial)

    kf = StratifiedKFold(3, shuffle=True, random_state=semente)
    
    # Avaliando o modelo por Validação cruzada 
    metrica = cross_val_score(
        modelo, 
        X_treino, 
        Y_treino, 
        scoring="f1", 
        cv=kf
    )

    return metrica.mean()

### $\bullet$ Criando um estudo do optuna

In [11]:
# Criando o estudo
study_knn = create_study(
    # Tipo de otimização
    direction="maximize",
    # Nome do estudo 
    study_name="Estudo KNN",
    # Salvando o estudo em um arquivo
    storage=f"sqlite:///{"Estudo KNN"}.db",
    # Recupera o progresso salvo do estudo
    load_if_exists=True,
)

[I 2025-10-23 14:31:48,524] A new study created in RDB with name: Estudo KNN


### $\bullet$ Determinando testes desejados 

In [12]:
# Modelo base sem tratamentos
study_knn.enqueue_trial(
    {
        "n_neighbors": 5, 
        "p": 2, 
        "weights": "uniform", 
        "normalization": None,
        "treatment": None
    }
)

# Modelo base com normalização padrão 
study_knn.enqueue_trial(
    {
        "n_neighbors": 5, 
        "p": 2, 
        "weights": "uniform", 
        "normalization": "standard",
        "treatment": None
    }
)

# Modelo base com normalização por mínimos e máximos 
study_knn.enqueue_trial(
    {
        "n_neighbors": 5, 
        "p": 2, 
        "weights": "uniform", 
        "normalization": "minmax",
        "treatment": None
    }
)

# Modelo base com normalização por máximo absoluto
study_knn.enqueue_trial(
    {
        "n_neighbors": 5, 
        "p": 2, 
        "weights": "uniform", 
        "normalization": "maxabs",
        "treatment": None
    }
)

# Modelo base com normalização padrão e PCA
study_knn.enqueue_trial(
    {
        "n_neighbors": 5, 
        "p": 2, 
        "weights": "uniform", 
        "normalization": "standard",
        "treatment": "pca",
        "pca_components": 38
    }
)



### $\bullet$ Criando a Função Objetivo Parcial

In [13]:
def parcial_knn(trial): 
    """Função que retorna a função objetivo"""
    return funcao_objetivo_knn(trial, x_treino, y_treino)


### $\bullet$ Otimizando os parâmetros 

In [31]:
# study_knn.optimize(parcial_knn, 100)

### $\bullet$ Resultados

In [15]:
resultado_knn = study_knn.best_trial
print(f"Número do melhor trial: {resultado_knn.number}")
print(f"Parâmetros do melhor trial: {resultado_knn.params}")

Número do melhor trial: 61
Parâmetros do melhor trial: {'n_neighbors': 1, 'p': 2, 'weights': 'distance', 'normalization': 'standard', 'treatment': 'pca', 'pca_components': 14}


## Otimizando um modelo Support Vector Machine Classifier 

### $\bullet$ Criando a Função para Instanciar o Modelo

In [16]:
def instanciador_svc(trial): 
    """Recebe um trial do optuna e retorna uma instância de um modelo com classificador SVC"""

    # Definindo os parâmetros do modelo 
    params = {
            "C": trial.suggest_float("C", 1e-3, 1e3, log=True),  
            "kernel": trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"]),
            "gamma": trial.suggest_categorical("gamma", ["scale", "auto"]),
            "degree": trial.suggest_int("degree", 1, 5), 
            "coef0": trial.suggest_float("coef0", 0.0, 1.0), 
        }

    # Lista de etapas para o pipeline 
    steps = []

    # Definindo a estratégia de normalização 
    normalization = trial.suggest_categorical("normalization", [None, "standard", "minmax", "maxabs"])

    # Adiciona normalização padrão
    if normalization == "standard": 
        steps.append(StandardScaler())
    # Adiciona normalização por máximos e mínimos
    elif normalization == "minmax": 
        steps.append(MinMaxScaler())
    # Adiciona normalização por máximo absoluto
    elif normalization == "maxabs": 
        steps.append(MaxAbsScaler())

    # Definindo estratégia de redução de dimensionalidade ou seleção de atributos 
    treatment = trial.suggest_categorical("treatment", [None, "pca"])

    # Adiciona tratamento PCA 
    if treatment == "pca": 
        # Definindo o número de componentes a serem mantidas pelo pca
        components = trial.suggest_int("pca_components", 2, 38)
        steps.append(PCA(n_components=components))

    # Instânciando o modelo
    modelo = SVC(**params)
    steps.append(modelo)

    # Criando o pipeline
    pipeline = make_pipeline(*steps)

    return pipeline

### $\bullet$ Criando a função objetivo do optuna

In [17]:
def funcao_objetivo_svc(trial, X_treino, Y_treino): 
    """Função a ser otimizada pelo Optuna"""
    # Instânciando o modelo 
    modelo = instanciador_svc(trial)

    kf = StratifiedKFold(3, shuffle=True, random_state=semente)
    
    # Avaliando o modelo por Validação cruzada 
    metrica = cross_val_score(
        modelo, 
        X_treino, 
        Y_treino, 
        scoring="f1", 
        cv=kf
    )

    return metrica.mean()

### $\bullet$ Criando um estudo do optuna

In [18]:
# Criando o estudo
study_svc = create_study(
    # Tipo de otimização
    direction="maximize",
    # Nome do estudo 
    study_name="Estudo SVC",
    # Salvando o estudo em um arquivo
    storage=f"sqlite:///{"Estudo SVC"}.db",
    # Recupera o progresso salvo do estudo
    load_if_exists=True,
)

[I 2025-10-23 14:32:42,251] A new study created in RDB with name: Estudo SVC


### $\bullet$ Determinando testes desejados 

In [19]:
# Modelo base sem tratamentos
study_svc.enqueue_trial(
    {
        "C": 1,  
        "kernel": "rbf",
        "gamma": "scale",
        "degree": 3, 
        "coef0": 0.0, 
        "normalization": None,
        "treatment": None
    }
)

# Modelo base com normalização padrão
study_svc.enqueue_trial(
    {
        "C": 1,  
        "kernel": "rbf",
        "gamma": "scale",
        "degree": 3, 
        "coef0": 0.0, 
        "normalization": "standard",
        "treatment": None
    }
)


# Modelo base com normalização por mínimos e máximos 
study_svc.enqueue_trial(
    {
        "C": 1,  
        "kernel": "rbf",
        "gamma": "scale",
        "degree": 3, 
        "coef0": 0.0, 
        "normalization": "minmax",
        "treatment": None
    }
)

# Modelo base com normalização por máximo absoluto
study_svc.enqueue_trial(
    {
        "C": 1,  
        "kernel": "rbf",
        "gamma": "scale",
        "degree": 3, 
        "coef0": 0.0, 
        "normalization": "maxabs",
        "treatment": None
    }
)

# Modelo base com normalização padrão e PCA
study_svc.enqueue_trial(
    {
        "C": 1,  
        "kernel": "rbf",
        "gamma": "scale",
        "degree": 3, 
        "coef0": 0.0, 
        "normalization": "standard",
        "treatment": "pca",
        "pca_components": 38
    }
)


### $\bullet$ Criando a Função Objetivo Parcial

In [20]:
def parcial_svc(trial): 
    """Função que retorna a função objetivo"""
    return funcao_objetivo_svc(trial, x_treino, y_treino)


### $\bullet$ Otimizando os parâmetros 

In [30]:
# study_svc.optimize(parcial_svc, 100)

### $\bullet$ Resultados

In [22]:
resultado_svc = study_svc.best_trial
print(f"Número do melhor trial: {resultado_svc.number}")
print(f"Parâmetros do melhor trial: {resultado_svc.params}")

Número do melhor trial: 28
Parâmetros do melhor trial: {'C': 8.069114099798579, 'kernel': 'linear', 'gamma': 'auto', 'degree': 1, 'coef0': 0.7363079808351389, 'normalization': 'minmax', 'treatment': None}


## Otimizando um modelo de Logistic Regression

### $\bullet$ Criando a Função para Instanciar o Modelo

In [23]:
def instanciador_lrc(trial): 
    """Recebe um trial do optuna e retorna uma instância de um modelo com classificador por Regressão Logística"""

    # Definindo os parâmetros do modelo 
    params = {
        "penalty": trial.suggest_categorical("penalty", ["l1", "l2"]),
        "C": trial.suggest_float("C", 0.01, 10.0, log=True),
        "class_weight": trial.suggest_categorical("class_weight", ["balanced", None]),
        "solver":"liblinear",
        "max_iter":5000,
        "random_state":semente
    }

    # Lista de etapas para o pipeline 
    steps = []

    # Definindo a estratégia de normalização 
    normalization = trial.suggest_categorical("normalization", [None, "standard", "minmax", "maxabs"])

    # Adiciona normalização padrão
    if normalization == "standard": 
        steps.append(StandardScaler())
    # Adiciona normalização por máximos e mínimos
    elif normalization == "minmax": 
        steps.append(MinMaxScaler())
    # Adiciona normalização por máximo absoluto
    elif normalization == "maxabs": 
        steps.append(MaxAbsScaler())

    # Definindo estratégia de redução de dimensionalidade ou seleção de atributos 
    treatment = trial.suggest_categorical("treatment", [None, "pca", "rfe"])

    # Adiciona tratamento PCA 
    if treatment == "pca": 
        # Definindo o número de componentes a serem mantidas pelo pca
        components = trial.suggest_int("pca_components", 2, 38)
        steps.append(PCA(n_components=components))

    # Adiciona tratamento RFE 
    elif treatment == "rfe": 
        # Definindo o estimador 
        estimator = LogisticRegression(**params)
        # Definindo o número de atributos a serem mantidos
        n_features_to_select = trial.suggest_int("rfe_features", 2, 38)
        steps.append(RFE(estimator=estimator, n_features_to_select=n_features_to_select))
        
    # Instânciando o modelo
    modelo = LogisticRegression(**params)
    steps.append(modelo)

    # Criando o pipeline
    pipeline = make_pipeline(*steps)

    return pipeline

### $\bullet$ Criando a função objetivo do optuna

In [24]:
def funcao_objetivo_lrc(trial, X_treino, Y_treino): 
    """Função a ser otimizada pelo Optuna"""
    # Instânciando o modelo 
    modelo = instanciador_lrc(trial)

    kf = StratifiedKFold(3, shuffle=True, random_state=semente)
    
    # Avaliando o modelo por Validação cruzada 
    metrica = cross_val_score(
        modelo, 
        X_treino, 
        Y_treino, 
        scoring="f1", 
        cv=kf
    )

    return metrica.mean()

### $\bullet$ Criando um estudo do optuna

In [25]:
# Criando o estudo
study_lrc = create_study(
    # Tipo de otimização
    direction="maximize",
    # Nome do estudo 
    study_name="Estudo LRC",
    # Salvando o estudo em um arquivo
    storage=f"sqlite:///{"Estudo LRC"}.db",
    # Recupera o progresso salvo do estudo
    load_if_exists=True,
)

[I 2025-10-23 14:33:32,958] A new study created in RDB with name: Estudo LRC


### $\bullet$ Determinando testes desejados 

In [26]:
# Modelo base sem tratamentos
study_lrc.enqueue_trial(
    {
        "penalty": "l2",  
        "C": 1,
        "class_weight": None,
        "normalization": None,
        "treatment": None
    }
)


# Modelo base com normalização padrão
study_lrc.enqueue_trial(
    {
        "penalty": "l2",  
        "C": 1, 
        "class_weight": None,
        "normalization": "standard",
        "treatment": None
    }
)


# Modelo base com normalização por mínimos e máximos 
study_lrc.enqueue_trial(
    {
        "penalty": "l2",  
        "C": 1,
        "class_weight": None, 
        "normalization": "minmax",
        "treatment": None
    }
)

# Modelo base com normalização por máximo absoluto
study_lrc.enqueue_trial(
    {
       "penalty": "l2",  
        "C": 1,
        "class_weight": None,
        "normalization": "maxabs",
        "treatment": None
    }
)

# Modelo base com normalização padrão e PCA
study_lrc.enqueue_trial(
    {
       "penalty": "l2",  
        "C": 1,
        "class_weight": None,
        "normalization": "standard",
        "treatment": "pca",
        "pca_components": 38
    }
)

# Modelo base com normalização padrão e RFE
study_lrc.enqueue_trial(
    {
       "penalty": "l2",  
        "C": 1,
        "class_weight": None,
        "normalization": "standard",
        "treatment": "rfe",
        "rfe_features": 19
    }
)

### $\bullet$ Criando a Função Objetivo Parcial

In [27]:
def parcial_lrc(trial): 
    """Função que retorna a função objetivo"""
    return funcao_objetivo_lrc(trial, x_treino, y_treino)

### $\bullet$ Otimizando os parâmetros 

In [32]:
# study_lrc.optimize(parcial_lrc, 100)

### $\bullet$ Otimizando os parâmetros 

In [33]:
resultado_lrc = study_lrc.best_trial
print(f"Número do melhor trial: {resultado_lrc.number}")
print(f"Parâmetros do melhor trial: {resultado_lrc.params}")

Número do melhor trial: 94
Parâmetros do melhor trial: {'penalty': 'l2', 'C': 2.7595008656881284, 'class_weight': None, 'normalization': None, 'treatment': 'pca', 'pca_components': 35}


# Teste dos modelos

# Referências